In [1]:
!pip install -q kaggle
!pip install -q timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00


In [93]:
import os
import time

from torchvision import models
from torchsummary import summary
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch
from torch.nn.utils import prune
from tqdm.auto import tqdm
from pathlib import Path
from timm.loss import LabelSmoothingCrossEntropy

# Methods

In [94]:
def get_model_size(model):
  dummy_input = torch.randn(1, 3, 224, 224)
  size = sum(torch.nn.utils.parameters_to_vector(model.parameters()).size() * 4) / (1024 * 1024)
  return size


def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', round(size/1024/1024, 2))
    os.remove('temp.p')


def performance_test(model, criterion, test_loader, device):
    model.to(device)
    model.eval()
    test_loss = 0
    total = 0
    correct_top1=0
    correct_top5=0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += criterion(outputs, labels).item()

            # Top-1 и Top-5 accuracy
            _, predicted = outputs.topk(5, dim=1)
            total += labels.size(0)
            correct_top1 += predicted[:, 0].eq(labels).sum().item()
            correct_top5 += predicted.eq(labels.view(-1, 1)).sum().item()

        test_loss /= len(test_loader)
        top1_accuracy = correct_top1 / total
        top5_accuracy = correct_top5 / total

        print(f'Loss: {test_loss:.2f}, acc@1: {top1_accuracy}, acc@5: {top5_accuracy}')


def speed_test(model, test_dataset, device, batch_size=32, n_repeats=1):

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    model.to(device)
    model.eval()


    with torch.no_grad():

        # warm-up
        for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

        start = time.time()
        for i in range(n_repeats):
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
        end = time.time()

    mean_time = (end-start)/n_repeats

    print(f'Mean time: {mean_time:.5f} s')
    print(f'Mean Speed: {len(test_dataset) / mean_time:.1f} img/s')


def model_result(model,
                 model_name,
                 criterion,
                 test_loader,
                 device,
                 batch_size=32,
                 n_repeats=1) -> None:
  print_size_of_model(model, model_name)
  performance_test(model, criterion, test_loader, device)
  speed_test(model, test_dataset, device)


# Load Data

**Warning! You need to dowload kaggle.json file from your kaggle account page, before load data**

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d gpiosenka/sports-classification

 99% 419M/424M [00:04<00:00, 115MB/s]
100% 424M/424M [00:04<00:00, 100MB/s]


In [6]:
!unzip -q sports-classification.zip

In [7]:
class_names = os.listdir('/content/train')
class_names.sort()
class_len = len(class_names)

In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

test_dataset = datasets.ImageFolder(Path('/content/test'), transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Load models

In [9]:
#load weights
!gdown 1ak2YuKl-481hC6uDKtlsonLFzIgz3zJ5


Downloading...
From: https://drive.google.com/uc?id=1ak2YuKl-481hC6uDKtlsonLFzIgz3zJ5
To: /content/swin_val_loss=0.91_acc1=0.982_acc5=1.0.pth
100% 350M/350M [00:06<00:00, 56.6MB/s]


In [217]:
model = models.swin_b()
model.head = nn.Sequential(
    nn.Linear(in_features=1024, out_features=512, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.3, inplace=False),
    nn.Linear(in_features=512, out_features=100, bias=True)
)
model.load_state_dict(torch.load('/content/swin_val_loss=0.91_acc1=0.982_acc5=1.0.pth'))

<All keys matched successfully>

In [211]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = LabelSmoothingCrossEntropy()

In [212]:
model_result(model, 'base_model', criterion, test_loader, device)

model:  base_model  	 Size (MB): 333.63


  0%|          | 0/16 [00:00<?, ?it/s]

Loss: 0.85, acc@1: 0.992, acc@5: 1.0
Mean time: 6.44871 s
Mean Speed: 77.5 img/s


In [10]:
# model = models.resnet101(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 170MB/s]


# Quantization

## Dynamic

In [ ]:
dynamic_model_q = torch.ao.quantization.quantize_dynamic(model, {torch.nn.Conv2d, torch.nn.Linear})
# dynamic_model_q.quant = torch.ao.quantization.QuantStub()
model_result(dynamic_model_q, 'dynamic_q', criterion, test_loader, device)

## Static

In [ ]:
model.eval()

In [17]:
model.qconfig = torch.ao.quantization.get_default_qconfig('x86')

In [24]:
backend = "fbgemm"  # x86 machine
torch.backends.quantized.engine = backend
model.qconfig = torch.ao.quantization.get_default_qconfig(backend)

In [19]:
model_prepared = torch.ao.quantization.prepare(model)

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
input_fp32 = torch.randn(4, 3, 224, 224)
model_prepared(input_fp32)


In [21]:
model_prepared_static_q = torch.ao.quantization.convert(model_prepared)
model_prepared_static_q.quant = torch.ao.quantization.QuantStub()

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:1209: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [ ]:
summary(model_prepared_static_q, (3, 224, 224))

# Prunning

In [265]:
# percent of pruning parameters (0.3 == 30%)
amount = 0.3

In [270]:
# load model
pruned_model = models.swin_b()
pruned_model.head = nn.Sequential(
    nn.Linear(in_features=1024, out_features=512, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.3, inplace=False),
    nn.Linear(in_features=512, out_features=100, bias=True)
)
pruned_model.load_state_dict(torch.load('/content/swin_val_loss=0.91_acc1=0.982_acc5=1.0.pth'))

<All keys matched successfully>

In [271]:
# create parameters to prune
parameters_to_prune = [(pruned_model.features[0][0], 'weight')]

for i in range(17):
  for name in ('weight', 'bias'):
    parameters_to_prune.append((pruned_model.features[5][i].mlp[0], name))
    parameters_to_prune.append((pruned_model.features[5][i].mlp[3], name))

parameters_to_prune = tuple(parameters_to_prune)

In [272]:
# get prunning
prune.global_unstructured(parameters=parameters_to_prune, pruning_method=prune.RandomUnstructured, amount=amount)

In [273]:
# remove original parameters
for (module, name) in parameters_to_prune:
  prune.remove(module, name)

In [274]:
model_result(pruned_model, 'pruned_model', criterion, test_loader, device)

model:  pruned_model  	 Size (MB): 333.63


  0%|          | 0/16 [00:00<?, ?it/s]

Loss: 1.41, acc@1: 0.838, acc@5: 0.958
Mean time: 6.27918 s
Mean Speed: 79.6 img/s


In [227]:
model_result(model, 'base_model', criterion, test_loader, device)

model:  base_model  	 Size (MB): 333.63


  0%|          | 0/16 [00:00<?, ?it/s]

Loss: 0.85, acc@1: 0.992, acc@5: 1.0
Mean time: 6.35271 s
Mean Speed: 78.7 img/s
